In [7]:
import heapq
import math
import time
import networkx as nx
import geopy.distance

# Define a function to calculate Haversine distance
def haversine_distance(coord1, coord2):
    return geopy.distance.geodesic(coord1, coord2).km

# Define Graph class with cities and distances
class CityGraph:
    def __init__(self):
        self.graph = nx.Graph()

    def add_city(self, city, latitude, longitude):
        self.graph.add_node(city, pos=(latitude, longitude))

    def add_road(self, city1, city2, distance):
        self.graph.add_edge(city1, city2, weight=distance * 2)  # Transition cost d(i, j) * 2

    def get_neighbors(self, city):
        return self.graph.neighbors(city)

    def get_distance(self, city1, city2):
        return self.graph[city1][city2]['weight']

    def heuristic(self, city, target):
        coord1 = self.graph.nodes[city]['pos']
        coord2 = self.graph.nodes[target]['pos']
        return haversine_distance(coord1, coord2)

# Implement Greedy Best-First Search
def greedy_best_first_search(graph, start1, start2):
    frontier = []
    heapq.heappush(frontier, (graph.heuristic(start1, start2), start1, start2))
    explored = set()
    nodes_generated = 0

    while frontier:
        _, current1, current2 = heapq.heappop(frontier)

        if current1 == current2:
            return current1, nodes_generated  # Found optimal meetup city

        explored.add((current1, current2))

        for neighbor1 in graph.get_neighbors(current1):
            for neighbor2 in graph.get_neighbors(current2):
                if (neighbor1, neighbor2) not in explored:
                    heapq.heappush(frontier, (graph.heuristic(neighbor1, neighbor2), neighbor1, neighbor2))
                    nodes_generated += 1

    return None, nodes_generated



/Users/soumikdutta/PycharmProjects/pythonAIProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [32]:
import geopandas as gpd

# Load the Taluka shapefile
gdf = gpd.read_file("../datasets/india_taluk.shp")
# gdf["NAME_1"].unique()




KeyError: 'West Bengal'

In [ ]:
# Extract Taluka name, latitude, and longitude
gdf["latitude"] = gdf.geometry.centroid.y
gdf["longitude"] = gdf.geometry.centroid.x
taluka_data = gdf[["NAME_3", "latitude", "longitude"]]

# Save as CSV for easy use
taluka_data.to_csv("../datasets/taluka_centers.csv", index=False)

In [10]:
import pandas as pd

taluka_df = pd.read_csv("taluka_centers.csv")
graph = CityGraph()
for _, row in taluka_df.iterrows():
    graph.add_city(row["NAME_3"], row["latitude"], row["longitude"])


In [22]:


# Run Greedy Best-First Search
start_city1 = "Gharghoda"
start_city2 = "Limbdi"

start_time = time.time()
meetup_city, nodes_generated = greedy_best_first_search(graph, start_city1, start_city2)
end_time = time.time()

print(f"Optimal Meetup City: {meetup_city}, Nodes Generated: {nodes_generated}, Time: {end_time - start_time:.4f} sec")

Optimal Meetup City: None, Nodes Generated: 0, Time: 0.0004 sec
